<a href="https://colab.research.google.com/github/thisisanu/online_deep_learning/blob/main/homework3/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()
!nvidia-smi


Mon Nov 10 00:11:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [55]:
import os
import shutil

# Ensure we are in /content
os.chdir("/content")
print("Current directory:", os.getcwd())

# If a folder named homework2 exists, remove it completely
if os.path.exists("online_deep_learning"):
    shutil.rmtree("online_deep_learning")
    print("Old online_deep_learning folder removed.")
else:
    print("No existing online_deep_learning folder found.")

!git clone https://github.com/thisisanu/online_deep_learning.git
%cd online_deep_learning/homework3


Current directory: /content
Old online_deep_learning folder removed.
Cloning into 'online_deep_learning'...
remote: Enumerating objects: 525, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 525 (delta 155), reused 27 (delta 6), pack-reused 295 (from 2)
Receiving objects: 100% (525/525), 3.19 MiB | 11.80 MiB/s, done.
Resolving deltas: 100% (342/342), done.
/content/online_deep_learning/homework3


In [56]:
!pip install numpy torch torchvision tqdm matplotlib

#download the datasets by running the following command from the main directory
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/classification_data.zip -o ./classification_data.zip && unzip -qo classification_data.zip
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/drive_data.zip -o ./drive_data.zip && unzip -qo drive_data.zip
!ls


bundle.py		 drive_data	 homework	  requirements.txt
classification_data	 drive_data.zip  Homework3.ipynb
classification_data.zip  grader		 README.md


The subtask requires creating a new Python file `train_classification.py` and adding initial setup code for imports, device configuration, data transformations, and data loading. This code block will create the file and include all the necessary components as specified in the instructions.



In [57]:
import os

# The content that will be written to train_classification.py
# This string contains all the necessary imports, function definitions,
# and the main execution block for the classification training setup.
file_content = """
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import os
import argparse
import torchvision.models as models
import time
import copy

# Device configuration
def setup_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    return device

# Data transformations
def get_data_transforms(image_size):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms

# Data loading
def get_data_loaders(data_dir, data_transforms, batch_size):
    image_datasets = {
        x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
        for x in ['train', 'val']
    }
    dataloaders = {
        x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2)
        for x in ['train', 'val']
    }
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes
    print(f"Found {dataset_sizes['train']} training images and {dataset_sizes['val']} validation images.")
    print(f"Classes: {class_names}")
    return dataloaders, dataset_sizes, class_names

# Define the Classification Model
def create_model(num_classes, pretrained=True):
    model = models.resnet18(pretrained=pretrained)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

# Define Loss Function and Optimizer
def setup_training_components(model, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return criterion, optimizer

# Training function
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model if it's the best validation accuracy so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Classification Training Setup')
    parser.add_argument('--data_dir', type=str, default='./classification_data', help='Directory with classification data')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size for data loaders')
    parser.add_argument('--image_size', type=int, default=224, help='Image size for resizing')
    parser.add_argument('--num_epochs', type=int, default=5, help='Number of epochs to train')
    args = parser.parse_args()

    device = setup_device()
    data_transforms = get_data_transforms(args.image_size)
    dataloaders, dataset_sizes, class_names = get_data_loaders(args.data_dir, data_transforms, args.batch_size)

    # Model, Loss, and Optimizer setup
    num_classes = len(class_names)
    model = create_model(num_classes).to(device)
    criterion, optimizer = setup_training_components(model)

    print(f"Model: {model.__class__.__name__} with {num_classes} output classes")
    print(f"Loss Function: {criterion.__class__.__name__}")
    print(f"Optimizer: {optimizer.__class__.__name__}")

    # Train the model
    model = train_model(model, dataloaders, criterion, optimizer, device, args.num_epochs)

    # Save the trained model
    os.makedirs('homework', exist_ok=True)
    torch.save(model.state_dict(), 'homework/classifier.th')
    print("Trained model saved to homework/classifier.th")

    print("Classification training setup complete.")
"""

# Write the content to the file
with open('train_classification.py', 'w') as f:
    f.write(file_content)
print("Created train_classification.py with initial setup code.")
print("Initial classification training setup components are ready in train_classification.py.")

Created train_classification.py with initial setup code.
Initial classification training setup components are ready in train_classification.py.


**Reasoning**:
The previous steps successfully created the `train_classification.py` file. Now, to verify its functionality and ensure the initial setup components (device config, data transforms, data loading) are correctly integrated, I will execute the script. This will confirm that the file is executable and performs the expected setup operations.



In [60]:
import os

!ls
print(os.listdir('./classification_data'))
print(os.listdir('./classification_data/train'))
print(os.listdir('./classification_data/val'))

import os
import shutil

data_dir = './classification_data'

# Loop through train and val directories
for subset in ['train', 'val']:
    subset_path = os.path.join(data_dir, subset)
    print(f"Processing {subset_path}...")

    # Get all files in the current subset directory
    image_files = [f for f in os.listdir(subset_path) if f.endswith('.jpg')]

    for image_file in image_files:
        # Extract class name from the filename (e.g., '16217' from '16217.jpg')
        class_name = os.path.splitext(image_file)[0]

        # Create the target directory for this class if it doesn't exist
        class_dir = os.path.join(subset_path, class_name)
        os.makedirs(class_dir, exist_ok=True)

        # Move the image to its class directory
        src_path = os.path.join(subset_path, image_file)
        dest_path = os.path.join(class_dir, image_file)
        shutil.move(src_path, dest_path)

    print(f"Finished organizing images in {subset_path}.")

print("Dataset preprocessing complete. Images are now organized into class subfolders.")

print("Verifying train_classification.py setup...")
!python train_classification.py --data_dir ./classification_data
print("Verification complete.")

bundle.py		 drive_data.zip   README.md
classification_data	 grader		  requirements.txt
classification_data.zip  homework	  train_classification.py
drive_data		 Homework3.ipynb
['val', 'train']
['15940', '03793', '15259', '11599', '10423', '02822', '13276', '16926', '11228', '00211', '12197', '20704', '06630', '09115', '19912', '15919', '16059', '15977', '09983', '18456', '01728', '07830', '10906', '13527', '16254', '20835', '07249', '12173', '01596', '15922', '09505', '10265', '01096', '17257', '12143', '14645', '17495', '04420', '17521', '20936', '19524', '12316', '09207', '03884', '01145', '10764', '18561', '00479', '12714', '14430', '03978', '07030', '18898', '01702', '02993', '11468', '16754', '03329', '06322', '13774', '18716', '11799', '18635', '18650', '01611', '12672', '00279', '00472', '17602', '04999', '10181', '05788', '03836', '06239', '18394', '19236', '18389', '04563', '00688', '20338', '07392', '01315', '16236', '01773', '08040', '08244', '01791', '01089', '04343', '1890

In [ ]:
!cat Homework3.ipynb

Now that the dataset is properly organized with class subfolders, let's re-run the `train_classification.py` script to verify that the data loading and setup components work correctly.

In [61]:
%ls
!git status

# Be careful not to "git add *" since there are datasets and logs
!git add homework/*.py
!git add *.py
!git commit -m "update"
!git push origin main

bundle.py                drive_data.zip   README.md
classification_data/     grader/          requirements.txt
classification_data.zip  homework/        train_classification.py
drive_data/              Homework3.ipynb
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	classification_data.zip
	classification_data/
	drive_data.zip
	drive_data/
	train_classification.py

nothing added to commit but untracked files present (use "git add" to track)
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!cat ../README.md

In [47]:
!pwd
!ls homework/
!python3 bundle.py homework ada3488

# optional: run the grader with your bundled homework
!python3 -m grader ada3488.zip -vv --disable_color

/content/online_deep_learning/homework3
classifier.th  __init__.py  models.py	 train_classification.py
datasets       metrics.py   __pycache__  train_detection.py
metrics.py
train_classification.py
train_detection.py
classifier.th
datasets
models.py
__init__.py
datasets/road_dataset.py
datasets/road_utils.py
datasets/road_transforms.py
datasets/classification_dataset.py
Submission created: /content/online_deep_learning/homework3/ada3488.zip 76.33 MB
Public grader loaded.
[DEBUG    00:00:000] Loading assignment
[DEBUG    00:00:635] Loading grader
[INFO     00:00:636] Classifier
[INFO     00:01:053]   - Predict                                            [ 10 / 10 ]
[WARNING  00:01:157]   - Accuracy                                           [ 0 / 25 ]
[WARNING  00:01:157] Failed to load classifier.th, make sure the default model arguments are set correctly
[WARNING  00:01:238]   - Accuracy: Extra Credit                             [ 0 / 2 ]
[WARNING  00:01:238] Failed to load classifier.t

In [ ]:
!cat README.md